<a href="https://colab.research.google.com/github/chaiai/deep-learning/blob/main/VGG_network_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [ ]:
# Define a small class MyClass
class MyClass:
    def __init__(self):
        # One class variable 'a' is set to 1
        self.var1 = 1

# Create an object of type MyClass()
my_obj = MyClass()

In [ ]:
my_obj.__dict__

{'var1': 1}

In [ ]:
vars(my_obj)

{'var1': 1}

In [ ]:
class Block(tf.keras.Model):
    def __init__(self, filters, kernel_size, repetitions, pool_size=2, strides=2):
        super(Block, self).__init__()
        self.filters = filters
        self.kernel_size = kernel_size
        self.repetitions = repetitions
        
        # Define a Conv2D_0, Conv2D_1, etc. based on number of repetitions
        for i in range(self.repetitions):
            
            # Define a Conv2D layer, specifying filters, kernel_size, activation, and padding.
            vars(self)[f'conv2D_{i}'] = tf.keras.layers.Conv2D(self.filters, self.kernel_size, activation='relu', padding='same')
            
        # Define the max pool layer that will be added after the Conv2D blocks
        self.max_pool = tf.keras.layers.MaxPool2D(pool_size, strides=strides)
        
    def call(self, inputs):
        # Access the class's Conv2D_0 layer.
        conv2D_0 = vars(self)['conv2D_0']
        
        # Connect the Conv2D_0 layer to inputs
        x = conv2D_0(inputs)
        
        # For the remaining Conv2D_i layers from 1 to 'repetitions', they will be connected to previous layer.
        for i in range(1, self.repetitions):
            # Access Conv2D_i by formatting the integer 'i'.
            conv2D_i = vars(self)[f'conv2D_{i}']
            
            # Use the Conv2D_i instance and connect it to the previous layer
            x = conv2D_i(x)
            
        # Finally, add the max_pool layer.
        max_pool = self.max_pool(x)
        
        return max_pool

In [ ]:
class MyVGG(tf.keras.Model):
    
    def __init__(self, num_classes):
        super(MyVGG, self).__init__()
        
        # Creating blocks of VGG with the following (filters, kernel_size, repetitions) configurations
        self.block_a = Block(filters=64, kernel_size=3, repetitions=2)
        self.block_b = Block(filters=128, kernel_size=3, repetitions=2)
        self.block_c = Block(filters=256, kernel_size=3, repetitions=3)
        self.block_d = Block(filters=512, kernel_size=3, repetitions=3)
        self.block_e = Block(filters=512, kernel_size=3, repetitions=3)
        
        # Classification head
        # Define a Flatten layer
        self.flatten = tf.keras.layers.Flatten()
        
        # Create a Dense layer with 256 units and ReLU activation function
        self.fc = tf.keras.layers.Dense(256, activation='relu')
        
        # Finally, add the softmax classifier using a Dense layer
        self.classifier = tf.keras.layers.Dense(num_classes, activation='softmax')
        
    def call(self, inputs):
        # Chain all the layers one after the other
        x = self.block_a(inputs)
        x = self.block_b(x)
        x = self.block_c(x)
        x = self.block_d(x)
        x = self.block_e(x)
        x = self.flatten(x)
        x = self.fc(x)
        x = self.classifier(x)
        return x

In [ ]:
dataset = tfds.load('cats_vs_dogs', split=tfds.Split.TRAIN, data_dir='data/')

# Initialize VGG with the number of classes 
vgg = MyVGG(num_classes=2)

# Compile with losses and metrics
vgg.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Define preprocessing function
def preprocess(features):
    # Resize and normalize
    image = tf.image.resize(features['image'], (224, 224))
    return tf.cast(image, tf.float32) / 255., features['label']

# Apply transformations to dataset
dataset = dataset.map(preprocess).batch(32)

# Train the custom VGG model
vgg.fit(dataset, epochs=50)

Epoch 1/50
727/727 [==============================] - 91s 124ms/step - loss: 0.6911 - accuracy: 0.5241
Epoch 2/50
727/727 [==============================] - 89s 123ms/step - loss: 0.6643 - accuracy: 0.6162
Epoch 3/50
727/727 [==============================] - 89s 122ms/step - loss: 0.6368 - accuracy: 0.6462
Epoch 4/50
727/727 [==============================] - 89s 122ms/step - loss: 0.6182 - accuracy: 0.6652
Epoch 5/50
727/727 [==============================] - 89s 122ms/step - loss: 0.6054 - accuracy: 0.6742
Epoch 6/50
727/727 [==============================] - 89s 122ms/step - loss: 0.5958 - accuracy: 0.6834
Epoch 7/50
727/727 [==============================] - 89s 122ms/step - loss: 0.5880 - accuracy: 0.6870
Epoch 8/50
727/727 [==============================] - 89s 122ms/step - loss: 0.5812 - accuracy: 0.6927
Epoch 9/50
727/727 [==============================] - 89s 122ms/step - loss: 0.5749 - accuracy: 0.6976
Epoch 10/50
727/727 [==============================] - 89s 122ms/step - l

In [ ]:
vgg.evaluate(dataset)

727/727 [==============================] - 90s 123ms/step - loss: 0.5566 - accuracy: 0.7136


[0.5566273331642151, 0.7136101722717285]